This file contains some functions to check that ICD, OPERTN, MAINSPEF and PROCODES codes match with the reference tables. Functions are tested on APC tables (positive and negative).

# Connection

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import numpy as np

%matplotlib inline

In [2]:
def connect():
    return pyodbc.connect(
        'DRIVER={/usr/lib/x86_64-linux-gnu/odbc/libtdsodbc.so}; SERVER=192.168.5.78;'
        'DATABASE=IMS-NHSDigital;UID=sa;PWD=2HsxpmkDLSlHlT;port=1433;'
        'TDS_Version=8.0;')
engine = sqlalchemy.create_engine('mssql+pyodbc://', creator=connect)
conn = engine.connect()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/sqlalchemy/connectors/pyodbc.py:92: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  "No driver name specified; "


# Check on PROCODES

Extract the unique set of hospitals (PROCODE) codes and provide patient count for each. Join this with the reference table for the PROCODE codes and flag those that do not match a code in the reference table.

In [263]:
#extract codes
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

#table_list=['dbo.APC_Linkage']

label_ID="PROCODE3"

unique_procodes=set()
for tablename in table_list:
    query= 'SELECT ' + label_ID  +' from ' + tablename
    print(query)
    procodes=pd.read_sql(query, conn)[label_ID]
    unique_procodes.update(procodes)
    print(len(unique_procodes))

#print(unique_procodes)
#I suggest to count for those not matching if we just care about them

SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201299
135
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201399
158
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201499
172
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201599
181
SELECT PROCODE3 from dbo.NIC58999_APC_Linkage_201613
188


In [264]:
#if I use the procode col the majority of codes are 5 digits,
#sometime endings with 00/01, sometimes with other letters or number
list(unique_procodes)[0:12]

['RXE',
 'RTG',
 'NT2',
 'RTX',
 'RGR',
 'RXQ',
 'RHM',
 'RR7',
 'RKB',
 'REN',
 'RAN',
 'RM3']

In [258]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimProvider.csv'
ref_label_procode='ProviderCode'
ref_procode=pd.read_csv(ref_table, delimiter="|")[ref_label_procode].unique()

/home/vsalvatelli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [259]:
#the majority of reference codes are 3digit codes, seems sensible to use the procode3 from HES tables
ref_procode[:30]

array(['------------', 'Missing', 'Unknown', '5A3', '5A4', '5A5', '5AT',
       '5C1', '5C2', '5C3', '5C4', '5C5', '5C9', '5CN', '5CQ', '5F1',
       '5FE', '5FL', '5GC', '5H8', '5J2', '5J9', '5JE', '5K3', '5K5',
       '5K7', '5KG', '5KL', '5L1', '5L3'], dtype=object)

In [260]:
len(set(ref_procode))

452

In [261]:
#some codes are not matching
unique_procodes.issubset(set(ref_procode))

False

In [265]:
#about 25% of coding is not matching with the full dataset
#if I use procode is about 90% ...
NM_procodes=[x for x in unique_procodes if x not in ref_procode]
print(len(NM_procodes)/len(unique_procodes))
print(NM_procodes)

0.010638297872340425
['8E3', '8A1']


In [84]:
ref_code_8=[x for x in ref_procode if x.startswith('8')]

In [85]:
ref_code_8

['8A101',
 '8E339',
 '8F689',
 '8HP32',
 '8HP46',
 '8HT55',
 '8HT56',
 '8J124',
 '8J244',
 '8J248',
 '8J285',
 '8G301']

In [86]:
#we might assume that 8E3 stands for 8A101 and 8E3 stands for 8E339
#for what we use these codes after? useful to join with Provider Caption (contains the name of the provider)?

In [151]:
#count occurencies for not matching codes
NM_df=pd.DataFrame({label_ID: [],'counts': []})
for code in NM_procodes:
    for tablename in table_list:
        query= 'SELECT ' + label_ID + ', COUNT(' + label_ID + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_ID + ' IS NOT NULL AND ' + label_ID +' = \'' + code + '\' GROUP BY ' + label_ID + ';'  
        #print(query)
        NM_df=NM_df.append(pd.read_sql(query, conn), ignore_index=True)

In [156]:
#grouping the counts from different tables
NM_df.groupby(label_ID)['counts'].sum()

PROCODE3
8A1    10.0
8E3     3.0
Name: counts, dtype: float64

In [ ]:
#how often these codes appear?
for tablename in table_list:
        query= 'SELECT COUNT(' + label_ID + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_ID + ' IS NOT NULL AND ' + label_ID +' = \'' + code + '\' GROUP BY ' + label_ID + ';'  
        
print(len(NM_procodes)/len(unique_procodes))

# Check TRETSPEF

Extract the unique set of main specialties (TRETSPEF) codes and provide count for each. Join this with the reference table for the MAINSPEF codes and flag those that do not match a code in the reference table.  **ACHTUNG** we changed from MAINSPEF to TRETSPEF, that is the actual code the consultant treated the patient for

In [266]:
#extract codes
#MAINSPEF is always a 3digit code or &(meaning not known)
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

label_spec="TRETSPEF"

unique_spec=set()
for tablename in table_list:
    query= 'SELECT ' + label_spec  +' from ' + tablename
    spec=pd.read_sql(query, conn)[label_spec]
    unique_spec.update(spec)
    #print(len(unique_mainspef))

In [267]:
#reading reference codes
#Missing and unknown labelled differently from &
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimSpecialties.csv'
ref_label_spec='SpecialtyCode'
ref_spec=pd.read_csv(ref_table, delimiter="|")[ref_label_spec].unique()

In [268]:
ref_spec

array(['-------------', 'Miss', 'Unk', '000', '100', '101', '102', '103',
       '104', '105', '106', '107', '108', '110', '120', '130', '140',
       '141', '142', '143', '144', '145', '146', '147', '148', '149',
       '150', '160', '161', '170', '171', '172', '173', '174', '180',
       '190', '191', '192', '199', '211', '212', '213', '214', '215',
       '216', '217', '218', '219', '220', '221', '222', '223', '241',
       '242', '251', '252', '253', '254', '255', '256', '257', '258',
       '259', '260', '261', '262', '263', '264', '280', '290', '291',
       '300', '301', '302', '303', '304', '305', '306', '307', '308',
       '309', '310', '311', '312', '313', '314', '315', '316', '317',
       '318', '319', '320', '321', '322', '323', '324', '325', '326',
       '327', '328', '329', '330', '331', '340', '341', '342', '343',
       '344', '345', '346', '350', '351', '352', '360', '361', '370',
       '371', '400', '401', '410', '420', '421', '422', '424', '430',
       '450', '4

In [269]:
unique_spec.issubset(set(ref_spec))

False

In [270]:
#& is unknow
NM_spec=[x for x in unique_spec if x not in ref_spec]
print(len(NM_spec)/len(unique_spec))
print(NM_spec)

0.011494252873563218
['&']


In [271]:
#This was for MAINSPEF
#No idea how to match 451...maybe 450 by proximity?
#ref_spec_4=[x for x in ref_spec if x.startswith('4')]
#ref_spec_4
#no reference codes containing the substring
#ref_spec_451=[x for x in ref_spec if '451' in x]
#ref_spec_451

In [272]:
#count occurencies for not matching codes
NM_df=pd.DataFrame({label_spec: [],'counts': []})
for code in NM_spec:
    for tablename in table_list:
        query= 'SELECT ' + label_spec + ', COUNT(' + label_spec + ') AS counts' + ' from ' + tablename + \
        ' WHERE '+ label_spec + ' IS NOT NULL AND ' + label_spec +' = \'' + code + '\' GROUP BY ' + label_spec + ';'  
        #print(query)
        NM_df=NM_df.append(pd.read_sql(query, conn), ignore_index=True)

In [273]:
#grouping the counts from different tables
NM_df.groupby(label_spec)['counts'].sum()

TRETSPEF
&    17
Name: counts, dtype: int64

# Check OPERTN

Extract the unique set of procedure codes (OPERTN) codes and provide count for each. Join this with the reference table for the procedure codes and flag those that do not match a code in the reference table.  

In [198]:
#extract codes
#OPERTN contains a 4digit code, a letter followed by 3 digits (it might be only 2 digits according to the book)
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

#TO BE GENERALIZED: we have to append the codes in ALL the opertn cols
label_proc="OPERTN_01"

unique_proc=set()
for tablename in table_list:
    query= 'SELECT ' + label_proc  +' from ' + tablename
    proc=pd.read_sql(query, conn)[label_proc]
    unique_proc.update(proc)
    #print(len(unique_mainspef))

In [250]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimProcedure.csv'
#4th digit is a variation of the main procedure
ref_label_proc='ProcedureLevel4Code'
#unique() is creating some issues, the same thing can be achieved with set
ref_proc=set(pd.read_csv(ref_table, delimiter="|", encoding = "ISO-8859-1")[ref_label_proc])

In [254]:
ref_proc

{'M201',
 'Z174',
 'R171',
 'T288',
 'J481',
 'J153',
 'T199',
 'L876',
 'H305',
 'W332',
 'T372',
 'U172',
 'V549',
 'P198',
 'Y072',
 'X058',
 'V574',
 'Z451',
 'K328',
 'N325',
 'C112',
 'Y015',
 'A445',
 'H071',
 'Y651',
 'Z264',
 'R251',
 'A202',
 'T839',
 'Z244',
 'L988',
 'A203',
 'E423',
 'M392',
 'C117',
 'T551',
 'Y494',
 'M649',
 'V017',
 'U152',
 'T461',
 'A682',
 'Q278',
 'Z444',
 'K092',
 'D207',
 'A032',
 'A312',
 'G154',
 'A439',
 'J382',
 'F062',
 'X211',
 'B101',
 'X031',
 'Q298',
 'Y358',
 'A622',
 'Z846',
 'Y393',
 'G384',
 'A229',
 'N082',
 'L829',
 'E892',
 'R201',
 'H052',
 'W642',
 'Q303',
 'C533',
 'Z552',
 'D141',
 'Z118',
 'W789',
 'T554',
 'W982',
 'K383',
 'H661',
 'M223',
 'X904',
 'V088',
 'H622',
 'F458',
 'K051',
 'L761',
 'Z161',
 'Z433',
 'T618',
 'T962',
 'J127',
 'V141',
 'T022',
 'C544',
 'O111',
 'J203',
 'N279',
 'M382',
 'M179',
 'X704',
 'K232',
 'T289',
 'U271',
 'Z586',
 'M581',
 'B229',
 'Z358',
 'V397',
 'V068',
 'V251',
 'S555',
 'Z895',
 

# Check ICD

In [222]:
#extract codes
table_list=['dbo.NIC58999_APC_Linkage_201299', \
            'dbo.NIC58999_APC_Linkage_201399', \
            'dbo.NIC58999_APC_Linkage_201499', \
            'dbo.NIC58999_APC_Linkage_201599',\
            'dbo.NIC58999_APC_Linkage_201613']

#TO BE GENERALIZED: we have to append the codes in ALL the opertn cols
label_diag="DIAG_01"

unique_diag=set()
for tablename in table_list:
    query= 'SELECT ' + label_diag  +' from ' + tablename
    diag=pd.read_sql(query, conn)[label_diag]
    unique_diag.update(diag)

In [ ]:
list(unique_diag)[:100]

In [225]:
#reading reference codes
ref_table='/home/vsalvatelli/SharedData/HES Reference Files/DimDiagnosis.csv'
#4th digit is a variation of the main procedure
ref_label_diag='DiagnosisLevel4Code'
ref_diag=pd.read_csv(ref_table, delimiter="|",encoding = "ISO-8859-1")[ref_label_diag].unique()

In [226]:
ref_diag

array(['-------------------', 'Missing', 'Unknown', ..., 'U848', 'U849',
       'U85X'], dtype=object)